In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

from bs4 import BeautifulSoup
import requests
import time

In [2]:
df_movies = pd.read_csv('movies.csv') 

In [3]:
df_ratings = pd.read_csv('ratings.csv')

In [4]:
df_links = pd.read_csv('links.csv', dtype = object)
df_links.fillna(value=0, inplace=True)

In [5]:
df_tags = pd.read_csv('tags.csv')

In [22]:
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
df_ratings

In [ ]:
df_links

In [ ]:
df_tags

In [6]:
header = {"accept-language": "es-ES"}

In [ ]:
def sacar_sinopsis(df_links, header):
    results = []
    for i, tmdbId in enumerate(df_links["tmdbId"]):
        link = "https://www.themoviedb.org/movie/" + str(tmdbId)
        r = requests.get(link, headers = header)
        bs = BeautifulSoup(r.content, "html.parser")
        try:
            sinopsis = bs.p.get_text()
            movie_id = df_links.at[i,"movieId"]
            results.append([movie_id,tmdbId,sinopsis])
        except:
            results.append([tmdbId,"No se encontro sinopsis"])
            continue
        time.sleep(1)
        print(f"{i+1}/{len(df_links)} completed")

    df_sinopsis = pd.DataFrame(results,columns=['movieId','tmdbId','sinopsis'])
    df_sinopsis.to_csv('sinopsis.csv', index=False)
    return df_sinopsis

In [ ]:
df_sinopsis = sacar_sinopsis(df_links, header)

In [7]:
link = "https://www.themoviedb.org/movie/" + df_links["tmdbId"][0]

In [8]:
r = requests.get(link, headers = header)

In [9]:
bs = BeautifulSoup(r.content, "html.parser")

In [10]:
print (bs.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="no-js" lang="es">
<head>
<title>Toy Story (1995) — The Movie Database (TMDB)</title>
<meta content="on" http-equiv="cleartype"/>
<meta charset="utf-8"/>
<meta content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast" name="keywords"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="Los juguetes de Andy, un niño de seis años, temen que haya llegado su hora y que un nuevo regalo de cumpleaños les sustituya en el corazón de su dueño. Woody, un vaquero que ha sido hasta ahora el juguete favorito de Andy, trata de tranquilizarlos hasta que aparece Buzz Lightyear, un héroe espacial dotado de todo tipo de avances tecnológicos. Woody es relegado a un segundo plano., pero su constante rivalidad se transformará en u

In [11]:
print(bs.p.get_text())

Los juguetes de Andy, un niño de seis años, temen que haya llegado su hora y que un nuevo regalo de cumpleaños les sustituya en el corazón de su dueño. Woody, un vaquero que ha sido hasta ahora el juguete favorito de Andy, trata de tranquilizarlos hasta que aparece Buzz Lightyear, un héroe espacial dotado de todo tipo de avances tecnológicos. Woody es relegado a un segundo plano., pero su constante rivalidad se transformará en una gran amistad cuando ambos se pierden en la ciudad sin saber cómo volver a casa.


In [12]:
df_sinopsis = pd.read_csv('sinopsis.csv')

In [13]:
df_usuario = pd.read_csv('Usuario_0.csv')

In [14]:
def juntarMoviesSinopsis(df_movies, df_sinopsis, movieId, movies_sinopsis):
    # Unir los dataframes en función de la columna 'movieId'
    df_movies_sinopsis = pd.merge(df_movies, df_sinopsis, on='movieId')

    # Eliminar columna tmdbId
    df_movies_sinopsis = df_movies_sinopsis.drop('tmdbId', axis=1)
    
    # Guardar el resultado en un nuevo archivo CSV
    df_movies_sinopsis.to_csv('movies_sinopsis.csv', index=False)    
    return df_movies_sinopsis

In [15]:
#Comprobar que crea y elimina y guarda en un fichero csv
df_movies_sinopsis = juntarMoviesSinopsis(df_movies, df_sinopsis, 'movieId', 'movies_sinopsis.csv')
print(df_movies_sinopsis)

      movieId                                      title  \
0           1                           Toy Story (1995)   
1           2                             Jumanji (1995)   
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
...       ...                                        ...   
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                           genres  \
0     Adventure|Animation|Children|Comedy|Fantasy   
1                      Adventure|Children|Fantasy   
2                                  Comedy|Romance   
3                            Comedy|Drama|Romance   
4             

In [51]:
df_movies = pd.read_csv('movies.csv')
df_usuario = pd.read_csv('Usuario_0.csv')

In [54]:
def sacarPelisNoVistasPorU0(df_movies, df_usuario, movieId, PNVU0):
    # Unir los dataframes en función de la columna 'movieId'
    merged_df = pd.merge(df_movies, df_usuario, on='movieId', how='outer', indicator=True)
    
    dfPNVU0 = merged_df[merged_df['_merge'].isin(['left_only', 'right_only'])]
    # Eliminar columna tmdbId
    dfPNVU0.drop(columns='_merge', inplace=True)
    dfPNVU0.drop('title_y', axis=1, inplace=True)
    dfPNVU0 = dfPNVU0.rename(columns={'title_x': 'title'})
    
    # Guardar el resultado en un nuevo archivo CSV
    dfPNVU0.to_csv("PNVU0.csv", index=False)    
    return dfPNVU0

In [55]:
#Comprobar que crea y elimina y guarda en un fichero csv
dfPNVU0 = sacarPelisNoVistasPorU0(df_movies, df_usuario, 'movieId', 'PNVU0.csv')
print(dfPNVU0)

      movieId                                      title  \
2           3                    Grumpier Old Men (1995)   
3           4                   Waiting to Exhale (1995)   
4           5         Father of the Bride Part II (1995)   
6           7                             Sabrina (1995)   
7           8                        Tom and Huck (1995)   
...       ...                                        ...   
9738   193581  Black Butler: Book of the Atlantic (2017)   
9739   193583               No Game No Life: Zero (2017)   
9740   193585                               Flint (2017)   
9741   193587        Bungo Stray Dogs: Dead Apple (2018)   
9742   193609        Andrew Dice Clay: Dice Rules (1991)   

                               genres  rating  
2                      Comedy|Romance     NaN  
3                Comedy|Drama|Romance     NaN  
4                              Comedy     NaN  
6                      Comedy|Romance     NaN  
7                  Adventure|Children  

In [56]:
dfPNVU0

,movieId,title,genres,rating
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN
6,7,Sabrina (1995),Comedy|Romance,NaN
7,8,Tom and Huck (1995),Adventure|Children,NaN
...,...,...,...,...
9738,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN
9739,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN
9740,193585,Flint (2017),Drama,NaN
9741,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN
